In [0]:
import numpy as np
from math import log2

In [0]:
class NodeDT:
  def __init__(self, X, y, feature_name):
    self.feature_name = feature_name
    self.X = X
    self.y = y
    self.is_leaf = False
    self.label = None
    self.used = []
  
  def entropy(self):
    n = len(self.y)
    sum_ = 0
    for i in np.unique(self.y):
      v = len(self.y[self.y == i])
      sum_ += -((v/n) * log2(v/n))
    return sum_
  
  def classification_error(self):
    pass

In [0]:
class DecisionTree:
  _metrics = {'ce': '_classification_error', 'ig': '_information_gain'}
  def __init__(self, max_depth=None, criterion='ig'):
    self.max_depth = max_depth
    self.criterion = criterion
    if self.criterion not in self._metrics.keys():
      self.criterion = 'ig'
    self.num_class = 0
    self.tree = None
    self.thresholds = {}
  
  def _is_numerical(self, feature):
    return len(np.unique(feature)) >= 100
  
  def _find_threshold(self, feature, y_train, num_class):
    assert len(num_class) == 2,
    best_threshold = 0.0
    max_exact_classification = 0.0
    is_positive_negative = False
    sorted_feature = sorted(np.unique(feature))
    for i in range(len(sorted_feature) - 1):
      threshold = (sorted_feature[i] + sorted_feature[i+1])/2
      left_partition = y_train[feature < threshold]
      right_partition = y_train[feature > threshold]
      negative_positive = ((len(left_partition[left_partition == 0]) + len(right_partition[right_partition == 1])) / len(feature))
      
      positive_negative = ((len(left_partition[left_partition == 1]) + len(right_partition[right_partition == 0])) / len(feature))
      
      is_positive_negative = positive_negative > negative_positive
      choose = positive_negative if is_positive_negative else negative_positive
      if max_exact_classification < choose:
        max_exact_classification = choose
        best_threshold = threshold
    return best_threshold, is_positive_negative 

  def _entropy(self, feature, node):
    entropy = 0
    categories = np.unique(feature)
    num_point = len(feature)
    for category in categories:
      num_category = len(feature[feature == category])
      for c in self.num_class:
        num_category_class = len(feature[np.logical_and(feature == category, node.y == c)])
        if num_category_class == 0:
          continue
        entropy += num_category / num_point * (-num_category_class / num_category * log2(num_category_class / num_category))
    return entropy
  